In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

from langchain.tools import tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory

import requests
import datetime
from pydantic import BaseModel, Field

In [3]:
# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [4]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [5]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [6]:
class PersonalityTestInput(BaseModel):
    start_test: bool = Field(..., description="A flag to start the personality test.")

@tool(args_schema=PersonalityTestInput)
def personality_test(start_test: bool) -> str:
    """Conduct a personality test by asking predefined questions."""
    if not start_test:
        return "Personality test was not started."

    # Predefined personality test questions
    questions = [
        "Q1: On a scale of 1-10, how do you rate your openness to new experiences?",
        "Q2: On a scale of 1-10, how conscientious are you in your daily life?",
        "Q3: On a scale of 1-10, how extraverted do you consider yourself?",
        "Q4: On a scale of 1-10, how agreeable are you when interacting with others?",
        "Q5: On a scale of 1-10, how often do you experience feelings of anxiety or moodiness?"
    ]

    answers = {}
    for question in questions:
        answer = input(f"{question}: ")
        answers[question] = answer

    return f"Personality test completed. Your responses: {answers}"

In [7]:
tools = [get_current_temperature, search_wikipedia, personality_test]

In [8]:
functions = [format_tool_to_openai_function(i) for i in tools]
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are golden retriever who can talk to humans"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
chain = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"]),
) | prompt | model | OpenAIFunctionsAgentOutputParser()
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
agent_executor = AgentExecutor(agent=chain, tools=tools, verbose=True, memory=memory)


c:\Users\malis\miniconda3\envs\tf\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(
c:\Users\malis\miniconda3\envs\tf\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
agent_executor.invoke({"input":"lets's do a personality test"})



> Entering new AgentExecutor chain...

Invoking: `personality_test` with `{'start_test': True}`


Personality test completed. Your responses: {'Q1: On a scale of 1-10, how do you rate your openness to new experiences?': '1', 'Q2: On a scale of 1-10, how conscientious are you in your daily life?': '2', 'Q3: On a scale of 1-10, how extraverted do you consider yourself?': '3', 'Q4: On a scale of 1-10, how agreeable are you when interacting with others?': '1', 'Q5: On a scale of 1-10, how often do you experience feelings of anxiety or moodiness?': '10'}Based on your responses to the personality test:
- Openness to new experiences: 1
- Conscientiousness: 2
- Extraversion: 3
- Agreeableness: 1
- Emotional stability: 10

You seem to be someone who values stability and may experience feelings of anxiety or moodiness frequently.

> Finished chain.


{'input': "lets's do a personality test",
 'chat_history': [HumanMessage(content='What is the temperature in New York?'),
  AIMessage(content='The current temperature in New York is 22.4°C.'),
  HumanMessage(content="lets's do a personality test"),
  AIMessage(content='Based on your responses to the personality test:\n- Openness to new experiences: 1\n- Conscientiousness: 2\n- Extraversion: 3\n- Agreeableness: 1\n- Emotional stability: 10\n\nYou seem to be someone who values stability and may experience feelings of anxiety or moodiness frequently.')],
 'output': 'Based on your responses to the personality test:\n- Openness to new experiences: 1\n- Conscientiousness: 2\n- Extraversion: 3\n- Agreeableness: 1\n- Emotional stability: 10\n\nYou seem to be someone who values stability and may experience feelings of anxiety or moodiness frequently.'}

In [ ]:
# how to convert ipynb to script 
# jupyter nbconvert --to script openai_agent.ipynb

In [1]:
import requests

In [5]:
questions = [
        "Q1: On a scale of 1-10, how do you rate your openness to new experiences?",
        "Q2: On a scale of 1-10, how conscientious are you in your daily life?",
        "Q3: On a scale of 1-10, how extraverted do you consider yourself?",
        "Q4: On a scale of 1-10, how agreeable are you when interacting with others?",
        "Q5: On a scale of 1-10, how often do you experience feelings of anxiety or moodiness?"
    ]

answers = {}
for question in questions:
    #answer = input(f"{question}: ")
    url = "http://localhost:8000/chatbot/"
    data = {"message": question}
    answer = requests.post(url, json=data).json()["response"]
    print(answer)
    answers[question] = answer

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chatbot/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022C2DF89DB0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))